In [ ]:
import shutil
from pathlib import Path
from utils import *
shutil.copy('1', Path(DEFAULT_CACHE_DIR+'/cached-files'))

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(
    model="chatglm3-6b",
    base_url="http://localhost:8765/v1/",
    api_key="EMPTY"# type:ignore
)

In [3]:
llm.invoke("hello")

AIMessage(content='Hello! How can I help you today?')

In [20]:
# 核心控制模块
from typing import Mapping
import openai
import os
import json
import logging
from pathlib import Path
from dotenv import load_dotenv
from Retrieval_qa import retrieval_auto_runner
from utils import *
from websearch2 import *
from websearch2 import get_customed_arxiv_search_tool
from Retrieval_qa import get_retrieval_tool
from global_var import get_global_value, set_global_value
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor, create_structured_chat_agent, load_tools
# from ChatGLM3 import ChatGLM3
from ChatGLM3Agent import *
from typing import List
from langchain.agents import Tool, AgentOutputParser
from langchain.prompts import StringPromptTemplate

logger = logging.getLogger('SDFDSFF')
GLM3_TEMPLATE = "You can answer using the tools, or answer directly using your knowledge without using the tools. Respond to the human as helpfully and accurately as possible.\nYou have access to the following tools:\n{tools}\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\nValid \"action\" values: \"Final Answer\" or  [{tool_names}]Provide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{{{\n  \"action\": $TOOL_NAME,\n  \"action_input\": $INPUT\n}}}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{{{\n  \"action\": \"Final Answer\",\n  \"action_input\": \"Final response to human\"\n}}}}\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.\nhistory: {history}\n\nQuestion: {input}\n\nThought: {agent_scratchpad}"
from langchain.agents import Tool, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from typing import List
from langchain.schema import AgentAction, AgentFinish
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[BaseTool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

class CustomOutputParser(AgentOutputParser):
    begin: bool = False
    def __init__(self):
        super().__init__()
        self.begin = True

    def parse(self, llm_output: str) -> AgentFinish | tuple[dict[str, str], str] | AgentAction:
        if self.begin:
            self.begin = False
            stop_words = ["Observation:"]
            min_index = len(llm_output)
            for stop_word in stop_words:
                index = llm_output.find(stop_word)
                if index != -1 and index < min_index:
                    min_index = index
                llm_output = llm_output[:min_index]

        if "Final Answer:" in llm_output:
            self.begin = True
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:", 1)[-1].strip()},
                log=llm_output,
            )
        parts = llm_output.split("Action:")
        if len(parts) < 2:
            return AgentFinish(
                return_values={"output": f"调用agent工具失败，该回答为大模型自身能力的回答:\n\n `{llm_output}`"},
                log=llm_output,
            )

        action = parts[1].split("Action Input:")[0].strip()
        action_input = parts[1].split("Action Input:")[1].strip()
        try:
            ans = AgentAction(
                tool=action,
                tool_input=action_input.strip(" ").strip('"'),
                log=llm_output
            )
            return ans
        except:
            return AgentFinish(
                return_values={"output": f"调用agent失败: `{llm_output}`"},
                log=llm_output,
            )

In [13]:

def call_agent(user_input:str, history:list[Mapping[str,str]]):
    tools_inst = [get_customed_arxiv_search_tool(load_all_available_meta=True, download=True), get_retrieval_tool()]
    endpoint_url = "http://localhost:8765/v1/chat/completions"
    llm = ChatOpenAI(
        model="chatglm3-6b",
        base_url="http://localhost:8765/v1/",
        api_key="EMPTY"# type:ignore
    )
    # prompt = hub.pull("hwchase17/structured-chat-agent")
    # prompt = CustomPromptTemplate(
    #         template=GLM3_TEMPLATE,
    #         tools=tools_inst,
    #         input_variables=["input", "intermediate_steps", "history"]
    #     )
    output_parser = CustomOutputParser()
    agent_executor = initialize_glm3_agent(
                llm=llm,
                tools=tools_inst,
                callback_manager=None,
                prompt=GLM3_TEMPLATE,
                input_variables=["input", "intermediate_steps", "history"],
                verbose=True,
            )
    agent_executor.invoke({"input": user_input, "history":[convert_dict_to_message(m) for m in history]})

In [ ]:
call_agent('帮我找有关AI的论文',[])

In [17]:
tools_inst = [get_customed_arxiv_search_tool(load_all_available_meta=True, download=True), get_retrieval_tool()]

In [19]:
print("\n".join([f"{tool.name}: {tool.description}" for tool in tools_inst]))

arxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.
retrieval: Retrieve the content of the cached papers and answer questions.


In [21]:
print(GLM3_TEMPLATE)

You can answer using the tools, or answer directly using your knowledge without using the tools. Respond to the human as helpfully and accurately as possible.
You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or  [{tool_names}]Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Obs

In [ ]:
StructuredGLM3ChatAgent